In [21]:
import pandas as pd
import sys, os
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from draw_analysis.team_draw_analysis import TeamDrawAnalysis
from munkres import Munkres
from enchant.utils import levenshtein
pd.options.display.max_rows = 100

In [2]:
df = pd.read_csv('team_market_value_web_scraping.csv')
df_copy = df.copy()
teams = pd.read_csv('team_id.csv')

In [3]:
df_copy.head()

,team,season,squad,age,foreigners,market_value,total_market_value
0,Málaga CF,2018,39,26.4,20,1220000,47400000
1,UD Las Palmas,2018,40,27.4,9,929000,37150000
2,Deportivo de La Coruña,2018,33,27.6,9,1120000,36850000
3,Granada CF,2018,30,26.2,6,1040000,31100000
4,Sporting Gijón,2018,36,25.0,10,819000,29500000


In [4]:
# lists of team names - Spanish alphabet and actual

teams_spanish_char = df_copy['team'].drop_duplicates().to_list()
teams_actual = teams['team'].to_list()

In [5]:
# match lists algorithm

def match_lists(l1, l2):
    # Compute a matrix of string distances for all combinations of
    # items in l1 and l2.
    matrix = [[levenshtein(i1, i2) for i2 in l2] for i1 in l1]

    # Now figure out what the global minimum distance between the
    # pairs is.
    indexes = Munkres().compute(matrix)
    for row, col in indexes:
        yield l1[row], l2[col]

In [6]:
gen = match_lists(teams_spanish_char, teams_actual)
team_names_matched = pd.DataFrame(gen)
team_names_matched

,0,1
0,Málaga CF,Malaga
1,UD Las Palmas,Las Palmas
2,Deportivo de La Coruña,Deportivo La Coruna
3,Granada CF,Granada CF
4,Sporting Gijón,Sporting Gijon
5,Cádiz CF,Cadiz
6,CA Osasuna,Osasuna
7,CD Tenerife,Tenerife
8,Albacete Balompié,Albacete
9,UD Almería,Almeria


#### Matching team names.

In [7]:
team_names_matched.set_index(keys=0, inplace=True)

In [8]:
df_copy['team'] = df_copy['team'].map(team_names_matched.to_dict()[1])

In [24]:
df_copy.to_csv('team_market_value.csv', index=False)